In [1]:
import numpy as np
import pandas as pd
import orca
import os; os.chdir('/home/amelia/ual_model_workspace/spring-2019-models/')
import warnings; warnings.simplefilter('ignore')
from urbansim.utils import misc
import pandana as pdna
import geopandas as gp
 
# from urbansim_templates import modelmanager as mm
# from urbansim_templates.models import LargeMultinomialLogitStep
# import os; os.chdir('/home/juan/ual_model_workspace/spring-2019-models/')
import warnings; warnings.simplefilter('ignore')
# from matplotlib import pyplot as plt
# import matplotlib.animation as animation
# import seaborn as sns

# from scripts import datasources, models, variables, utils
# scripts = '~/ual_model_workspace/spring-2019-models/scripts'

In [2]:
from scripts import datasources, models, variables, utils

Registering model step 'auto_ownership'
Registering model step 'TOD_choice'
Registering model step 'primary_mode_choice'
Registering model step 'WLCM'


In [3]:
bay_counties = [1, 13, 97, 95, 81, 85, 75, 41, 55]

In [4]:
orca.run(['initialize_network_small', 'initialize_network_walk','impute_missing_skims']) 

Running step 'initialize_network_small'
Time to execute step 'initialize_network_small': 0.00 s
Running step 'initialize_network_walk'
Time to execute step 'initialize_network_walk': 0.00 s
Running step 'impute_missing_skims'
Time to execute step 'impute_missing_skims': 96.66 s
Total time to execute iteration 1 with iteration value None: 96.66 s


In [ ]:
os.listdir('/home/data/CHTS_csv_format/data')

In [5]:
place = pd.read_csv('/home/data/CHTS_csv_format/data/Deliv_PLACE.csv')
person = pd.read_csv('/home/data/CHTS_csv_format/data/Deliv_PER.csv')
activity = pd.read_csv('/home/data/CHTS_csv_format/data/Deliv_ACTIVITY.csv')
schools = pd.read_csv('/home/juan/ual_model_workspace/spring-2019-models/notebooks-juan/schools.csv').rename({"parcel_id": "school_parcel_id"}, axis=1)
students = pd.read_csv('/home/juan/ual_model_workspace/spring-2019-models/notebooks-juan/students_with_school_id.csv')
parcels = orca.get_table('parcels').to_frame()
tracts = gp.read_file('/home/amelia/ual_model_workspace/spring-2019-models/notebooks-amelia/files/San_Francisco_Bay_Region_2010_Census_Tracts.shp')
TAZs = gp.read_file('/home/amelia/ual_model_workspace/spring-2019-models/notebooks-amelia/files/Transportation_Analysis_Zones.shp')





In [6]:
TAZs["centroid"] = TAZs["geometry"].apply(lambda x: x.centroid)
TAZs["Longitude"] = TAZs["centroid"].apply(lambda x: x.x)
TAZs["Latitude"] = TAZs["centroid"].apply(lambda x: x.y)
tracts["centroid"] = tracts["geometry"].apply(lambda x: x.centroid)
tracts["Longitude"] = tracts["centroid"].apply(lambda x: x.x)
tracts["Latitude"] = tracts["centroid"].apply(lambda x: x.y)


,objectid,taz1454,district,county,gacres,Shape__Are,Shape__Len,geometry,centroid,Longitude,Latitude
0,1,566,11,Santa Clara,1708.683056,0.000444,0.097550,"POLYGON ((-121.868062086 37.2871988910001, -12...",POINT (-121.8576467424699 37.28927060084754),-121.857647,37.289271
1,2,565,11,Santa Clara,486.200669,0.000126,0.054171,"POLYGON ((-121.858769084 37.2802438910001, -12...",POINT (-121.8533379635371 37.27758214291228),-121.853338,37.277582
2,3,573,11,Santa Clara,538.069933,0.000140,0.057053,"POLYGON ((-121.812377078 37.2888908950001, -12...",POINT (-121.8155788471425 37.29462625606124),-121.815579,37.294626
3,4,571,11,Santa Clara,450.407906,0.000117,0.052042,"POLYGON ((-121.805965075 37.2773018950001, -12...",POINT (-121.8156464942156 37.28068244491108),-121.815646,37.280682
4,5,576,11,Santa Clara,423.273023,0.000110,0.043438,"POLYGON ((-121.827876081 37.3085458960001, -12...",POINT (-121.8326348863136 37.31193308346154),-121.832635,37.311933


#### Filter for just the students in the CHTS persons table

In [13]:
chts_stu = person.loc[person['STUDE'].isin([1, 2])]
print(len(person))
print(len(chts_stu))

109113
27331


#### In Place table: create unique record identifiers by person, place and hh

In [8]:
place['HHPER'] = place['SAMPN'].map(str) + place['PERNO'].map(str) #sampn is hh identifier, perno is the person num within
#the household

place['HHPERPLA'] = place['HHPER'].map(str) + place['PLANO'].map(str)

# place.loc[ (place['Start_time'] > 24), 'Start_time'] = place['Start_time'] - 24

# place.loc[ (place['End_time'] > 24),'End_time'] = place['End_time'] - 24
place1 = place[['SAMPN', 'HHPER','HHPERPLA','MODE',"ARR_HR", "ARR_MIN", "DEP_HR", "DEP_MIN",'PNAME']]
place1.head()

,SAMPN,HHPER,HHPERPLA,MODE,ARR_HR,ARR_MIN,DEP_HR,DEP_MIN,PNAME
0,1031985,10319851,103198511,NaN,3,0,8,40,HOME
1,1031985,10319851,103198512,6.0,9,2,12,53,BENICIA SENIOR CITIZEN CENTER
2,1031985,10319851,103198513,6.0,13,13,2,59,HOME
3,1031985,10319852,103198521,NaN,3,0,9,16,HOME
4,1031985,10319852,103198522,5.0,9,26,10,43,LOMA VISTA ELEMENTARY SCHOOL


#### In activity table: create unique record identifiers by person, place and hh id

In [41]:
# activity['HHPER'] = activity['SAMPN'].map(str) + activity['PERNO'].map(str)
# activity['HHPERPLA'] = activity['HHPER'].map(str) + activity['PLANO'].map(str)
# activity['count'] = activity.groupby('HHPERPLA').cumcount() +1 #group by unique person-hh-place key, to account 
#for the same person going to the same place but doing different activities, and therefore having different
#trip purposes. cumcount() +1 numbers these different activities for the same person-place trip
activity2 = activity.pivot(index='HHPERPLA', columns='count', values='APURP').reset_index()
# #then take these person-place grouped entries and articulate the different activity purposes (up to 3)
activity2.columns = ['HHPERPLA', 'APURP1', 'APURP2', 'APURP3']
activity2["HHPER"] = activity2["HHPERPLA"].apply(lambda x: x[:-1])
activity2.head()

,HHPERPLA,APURP1,APURP2,APURP3,HHPER
0,103198511,1.0,2.0,1.0,10319851
1,103198512,37.0,NaN,NaN,10319851
2,103198513,2.0,7.0,8.0,10319851
3,103198521,1.0,2.0,7.0,10319852
4,103198522,33.0,NaN,NaN,10319852


Classify trip purpose as a school trip or not- if at least one of the 3 trip purposes is for school/studying (5), 
school/classroom/laboratory (17), meals at school/college (18), and also After-school or non-class-related
sports/physical activities (19),  All other after-school or non-class related activities (library, music rehearsal, clubs, etc.) (20). 

In [67]:
#place column is "school" if at least one of the 3 trip purpose columns is 5 (school/study trip purpose)
school_purposes = [5.0, 17.0, 18.0, 19.0, 20.0, 5, 17, 18, 19, 20]
work_purposes = [9.0, 10.0, 16.0, 9, 10, 16]

def assign_trip_type(row):
    if ((row["APURP1"] in school_purposes) | (row["APURP2"] in school_purposes) |
        (row["APURP3"] in school_purposes)):
         return "school"
    
    elif ((row["APURP1"] in work_purposes) | (row["APURP2"] in work_purposes) | 
          (row["APURP3"] in work_purposes)):
        return "work"
    
    else:
        return "other"
    

activity2['trip_type'] = activity2.apply(lambda row: assign_trip_type(row), axis=1)
activity2["trip_origin"] = activity2.groupby('HHPER', group_keys=False).trip_type.shift(-1)




,HHPERPLA,APURP1,APURP2,APURP3,HHPER,trip_type,trip_origin
0,103198511,1.0,2.0,1.0,10319851,other,other
1,103198512,37.0,NaN,NaN,10319851,other,other
2,103198513,2.0,7.0,8.0,10319851,other,NaN
3,103198521,1.0,2.0,7.0,10319852,other,other
4,103198522,33.0,NaN,NaN,10319852,other,other
5,103198523,8.0,2.0,1.0,10319852,other,NaN
6,103203611,1.0,NaN,NaN,10320361,other,other
7,103203612,34.0,31.0,34.0,10320361,other,other
8,103203613,1.0,NaN,NaN,10320361,other,other
9,103203614,27.0,NaN,NaN,10320361,other,other


In [76]:
place2 = pd.merge(place1, activity2, on='HHPERPLA').drop(["HHPER_y"], axis=1)
place2 = place2.rename({"HHPER_x": "HHPER"}, axis=1)
place2.head()

,SAMPN,HHPER,HHPERPLA,MODE,ARR_HR,ARR_MIN,DEP_HR,DEP_MIN,PNAME,APURP1,APURP2,APURP3,trip_type,trip_origin
0,1031985,10319851,103198511,NaN,3,0,8,40,HOME,1.0,2.0,1.0,other,other
1,1031985,10319851,103198512,6.0,9,2,12,53,BENICIA SENIOR CITIZEN CENTER,37.0,NaN,NaN,other,other
2,1031985,10319851,103198513,6.0,13,13,2,59,HOME,2.0,7.0,8.0,other,NaN
3,1031985,10319852,103198521,NaN,3,0,9,16,HOME,1.0,2.0,7.0,other,other
4,1031985,10319852,103198522,5.0,9,26,10,43,LOMA VISTA ELEMENTARY SCHOOL,33.0,NaN,NaN,other,other


merge with the students from chts persons data frame to filter for just the trips by students and with the available geo info of the schools that they go to

In [11]:
od = pd.read_csv("/home/data/fall_2018/chts_persons_w_zone_ids.csv")
#create unique hh-person-place identifiers from chts od pair data
od['HHPER'] = od['SAMPN'].map(str) + od['PERNO'].map(str)
od = od[['HHPER','zone_id_home','HXCORD','HYCORD']]
od.head()

,HHPER,zone_id_home,HXCORD,HYCORD
0,10319851,1247,-122.182785,38.161600
1,10319852,1247,-122.182785,38.161600
2,10339441,168,-122.470070,37.718091
3,10352741,424,-122.017274,37.388281
4,10352742,424,-122.017274,37.388281


In [77]:
# sampn is hh identifier, perno is the person num within the hh, sp HHPER is a unique person identifier

chts_stu['HHPER'] = chts_stu['SAMPN'].map(str) + chts_stu['PERNO'].map(str) 
chts_stu.columns[-100:]
# students dataframe- this is just the students in the chts with information about what school they go to
chts_stu1 = chts_stu[['HHPER','STUDE', 'SCHOL', 'SLOC', 'SCITY', 'STRACT','SSTAT', 'SZIP', 'SMODE']]

chts_stu1.head()


,HHPER,STUDE,SCHOL,SLOC,SCITY,STRACT,SSTAT,SZIP,SMODE
4,10320363,1,4.0,2.0,SAN DIEGO,17030.0,CA,92127,7.0
5,10320364,1,3.0,2.0,SAN DIEGO,17032.0,CA,92127,7.0
6,10320365,1,3.0,2.0,SAN DIEGO,17030.0,CA,92127,4.0
9,10320533,1,3.0,2.0,MERCED,1602.0,CA,95341,1.0
10,10320534,1,3.0,2.0,MERCED,1602.0,CA,95341,1.0


In [56]:
# #sampn is hh identifier, perno is the person num within the hh, sp HHPER is a unique person identifier

# students['HHPER'] = students['SAMPN'].map(str) + students['PERNO'].map(str) 

# #students dataframe- this is just the students in the chts with information about what school they go to
# students1 = students[["SAMPN", "HHPER","PERNO","SMODE", 'parcel_id_home', 'zone_id_home', 'school_id']]

# display(students1.head())

# print("unique hhper ids for students in place2 df: ", len(students1)) 
# print("non unique hh per ids in place2 df: ", len(place2)) 
# print("unique hh per ids in place2 df: ", len(place2["HHPER"].unique())) 

#### Assign locations to home and school locations

In [80]:
school_home_trips = pd.merge(place2, chts_stu1, on='HHPER')
display(school_home_trips.head())
print("non unique hhper ids of students matched with schools:", len(school_home_trips))
print("unique hhper ids of students matched with schools: ", len(school_home_trips["HHPER"].unique()))

,SAMPN,HHPER,HHPERPLA,MODE,ARR_HR,ARR_MIN,DEP_HR,DEP_MIN,PNAME,APURP1,...,trip_type,trip_origin,STUDE,SCHOL,SLOC,SCITY,STRACT,SSTAT,SZIP,SMODE
0,1032036,10320363,103203631,NaN,3,0,11,10,HOME,1.0,...,other,other,1,4.0,2.0,SAN DIEGO,17030.0,CA,92127,7.0
1,1032036,10320363,103203632,6.0,11,30,11,31,BERNARDO HEIGHTS MIDDLE SCHOOL,21.0,...,other,other,1,4.0,2.0,SAN DIEGO,17030.0,CA,92127,7.0
2,1032036,10320363,103203633,6.0,11,45,11,52,LITTLE CAESARS PIZZA,31.0,...,other,other,1,4.0,2.0,SAN DIEGO,17030.0,CA,92127,7.0
3,1032036,10320363,103203634,6.0,12,0,15,30,BERNARDO HEIGHTS MIDDLE SCHOOL,34.0,...,other,other,1,4.0,2.0,SAN DIEGO,17030.0,CA,92127,7.0
4,1032036,10320363,103203635,6.0,15,50,2,59,HOME,1.0,...,other,NaN,1,4.0,2.0,SAN DIEGO,17030.0,CA,92127,7.0


non unique hhper ids of students matched with schools: 109777
unique hhper ids of students matched with schools:  27184


#### merge with census tracts dataset to get centroid and county information

In [20]:
display(chts_stu1.head())
display(tracts.head())

,HHPER,STUDE,SCHOL,SLOC,SCITY,STRACT,SSTAT,SZIP,SMODE
4,10320363,1,4.0,2.0,SAN DIEGO,17030.0,CA,92127,7.0
5,10320364,1,3.0,2.0,SAN DIEGO,17032.0,CA,92127,7.0
6,10320365,1,3.0,2.0,SAN DIEGO,17030.0,CA,92127,4.0
9,10320533,1,3.0,2.0,MERCED,1602.0,CA,95341,1.0
10,10320534,1,3.0,2.0,MERCED,1602.0,CA,95341,1.0


,objectid,fipst,fipco,trctid,trctname,Shape__Are,Shape__Len,geometry,centroid,Longitude,Latitude
0,1,06,001,06001400100,Census Tract 4001,0.000706,0.144231,"POLYGON ((-122.24693315 37.885443009, -122.246...",POINT (-122.2319597399706 37.86763312517441),-122.231960,37.867633
1,2,06,001,06001400200,Census Tract 4002,0.000060,0.041978,"POLYGON ((-122.257936147 37.842611004, -122.25...",POINT (-122.2496046039934 37.84814279705468),-122.249605,37.848143
2,3,06,001,06001400300,Census Tract 4003,0.000113,0.056027,"POLYGON ((-122.265644148 37.837641003, -122.26...",POINT (-122.2544533536005 37.84060204539164),-122.254453,37.840602
3,4,06,001,06001400400,Census Tract 4004,0.000073,0.037274,"POLYGON ((-122.261839148 37.841628004, -122.26...",POINT (-122.257460839504 37.84828587664912),-122.257461,37.848286
4,5,06,001,06001400500,Census Tract 4005,0.000060,0.033135,"POLYGON ((-122.26952015 37.8485870040001, -122...",POINT (-122.264740790391 37.84854626570166),-122.264741,37.848546


#### Approximate school location by matching the centroid of the census tract with its block id, and relating that to a TAZ. school name, block id, and lat lon information have been redacted from CHTS so this is only an approximation of the location with the information available

In [21]:
#get school location from school_id
from sklearn.neighbors import BallTree

def parcel_id(data):
    ''' Assigns a parcel ID given a x and y coordinate
    Input: DataFrame with x and y coordinates
    Output: Array-like Parcel_ID'''
    #Transforming Latitude/Longitude to radians
    parcels_rad = np.deg2rad(parcels[['y', 'x']])
    data_rad = np.deg2rad(data)
    
    tree = BallTree(parcels_rad, metric='haversine')
    
    dists, idxs = tree.query(data_rad, return_distance=True)
    
    return parcels.iloc[idxs[:,0]].index

def get_zone_id(parcel_id):
    '''gets the zone_id (TAZ) of school and home locations by the parcel id'''
    try: 
#         print(type(parcel_id))
        zone_id = parcels.loc[parcel_id]["zone_id"]
#         reset_parcels = parcels.reset_index()
#         parcel_id = list((reset_parcels[reset_parcels["parcel_id"]])["zone_id"])
#         parcel_id = list(parcels.loc[parcel_id,'zone_id'])

    except Exception as e:
        zone_id = np.nan
#         print(e)
        
    return zone_id

In [24]:
chts_stu1.head()

,HHPER,STUDE,SCHOL,SLOC,SCITY,STRACT,SSTAT,SZIP,SMODE
4,10320363,1,4.0,2.0,SAN DIEGO,17030.0,CA,92127,7.0
5,10320364,1,3.0,2.0,SAN DIEGO,17032.0,CA,92127,7.0
6,10320365,1,3.0,2.0,SAN DIEGO,17030.0,CA,92127,4.0
9,10320533,1,3.0,2.0,MERCED,1602.0,CA,95341,1.0
10,10320534,1,3.0,2.0,MERCED,1602.0,CA,95341,1.0


In [25]:
tracts["TRACT"] = tracts["trctid"].apply(lambda x: float(x[4:])/100)
school_home_trips_tract = pd.merge(tracts, school_home_trips, right_on='STRACT', left_on='TRACT')
print(school_home_trips_tract.columns[-100:])
school_home_trips_tract.head()

Index(['objectid', 'fipst', 'fipco', 'trctid', 'trctname', 'Shape__Are',
       'Shape__Len', 'geometry', 'centroid', 'Longitude', 'Latitude', 'TRACT',
       'SAMPN', 'HHPER', 'HHPERPLA', 'MODE', 'ARR_HR', 'ARR_MIN', 'DEP_HR',
       'DEP_MIN', 'PNAME', 'APURP1', 'APURP2', 'APURP3', 'trip_type', 'STUDE',
       'SCHOL', 'SLOC', 'SCITY', 'STRACT', 'SSTAT', 'SZIP', 'SMODE'],
      dtype='object')


,objectid,fipst,fipco,trctid,trctname,Shape__Are,Shape__Len,geometry,centroid,Longitude,...,APURP3,trip_type,STUDE,SCHOL,SLOC,SCITY,STRACT,SSTAT,SZIP,SMODE
0,2,06,001,06001400200,Census Tract 4002,0.000060,0.041978,"POLYGON ((-122.257936147 37.842611004, -122.25...",POINT (-122.2496046039934 37.84814279705468),-122.249605,...,NaN,school,1,3.0,2.0,LA PRESA,14002.0,CA,91977,6.0
1,109,06,001,06001410400,Census Tract 4104,0.000072,0.035511,"POLYGON ((-122.169225122 37.742768002, -122.16...",POINT (-122.1621872388229 37.74082109234572),-122.162187,...,NaN,school,1,7.0,NaN,SEASIDE,14104.0,CA,93955,15.0
2,109,06,001,06001410400,Census Tract 4104,0.000072,0.035511,"POLYGON ((-122.169225122 37.742768002, -122.16...",POINT (-122.1621872388229 37.74082109234572),-122.162187,...,NaN,school,2,7.0,NaN,SEASIDE,14104.0,CA,93955,5.0
3,109,06,001,06001410400,Census Tract 4104,0.000072,0.035511,"POLYGON ((-122.169225122 37.742768002, -122.16...",POINT (-122.1621872388229 37.74082109234572),-122.162187,...,5.0,school,1,8.0,NaN,SEASIDE,14104.0,CA,93955,5.0
4,109,06,001,06001410400,Census Tract 4104,0.000072,0.035511,"POLYGON ((-122.169225122 37.742768002, -122.16...",POINT (-122.1621872388229 37.74082109234572),-122.162187,...,NaN,school,1,8.0,NaN,SEASIDE,14104.0,CA,93955,5.0


In [28]:
school_home_trips_tract["school_parcel_id*"] = parcel_id(school_home_trips_tract[["Latitude", "Longitude"]])
school_home_trips_tract["school_taz*"] = school_home_trips_tract["school_parcel_id*"].apply(lambda x: get_zone_id(x))
school_home_trips_tract.head(10)


,objectid,fipst,fipco,trctid,trctname,Shape__Are,Shape__Len,geometry,centroid,Longitude,...,STUDE,SCHOL,SLOC,SCITY,STRACT,SSTAT,SZIP,SMODE,school_parcel_id*,school_taz*
0,2,06,001,06001400200,Census Tract 4002,0.000060,0.041978,"POLYGON ((-122.257936147 37.842611004, -122.25...",POINT (-122.2496046039934 37.84814279705468),-122.249605,...,1,3.0,2.0,LA PRESA,14002.0,CA,91977,6.0,85784,999
1,109,06,001,06001410400,Census Tract 4104,0.000072,0.035511,"POLYGON ((-122.169225122 37.742768002, -122.16...",POINT (-122.1621872388229 37.74082109234572),-122.162187,...,1,7.0,NaN,SEASIDE,14104.0,CA,93955,15.0,80358,879
2,109,06,001,06001410400,Census Tract 4104,0.000072,0.035511,"POLYGON ((-122.169225122 37.742768002, -122.16...",POINT (-122.1621872388229 37.74082109234572),-122.162187,...,2,7.0,NaN,SEASIDE,14104.0,CA,93955,5.0,80358,879
3,109,06,001,06001410400,Census Tract 4104,0.000072,0.035511,"POLYGON ((-122.169225122 37.742768002, -122.16...",POINT (-122.1621872388229 37.74082109234572),-122.162187,...,1,8.0,NaN,SEASIDE,14104.0,CA,93955,5.0,80358,879
4,109,06,001,06001410400,Census Tract 4104,0.000072,0.035511,"POLYGON ((-122.169225122 37.742768002, -122.16...",POINT (-122.1621872388229 37.74082109234572),-122.162187,...,1,8.0,NaN,SEASIDE,14104.0,CA,93955,5.0,80358,879
5,109,06,001,06001410400,Census Tract 4104,0.000072,0.035511,"POLYGON ((-122.169225122 37.742768002, -122.16...",POINT (-122.1621872388229 37.74082109234572),-122.162187,...,1,8.0,NaN,SEASIDE,14104.0,CA,93955,5.0,80358,879
6,109,06,001,06001410400,Census Tract 4104,0.000072,0.035511,"POLYGON ((-122.169225122 37.742768002, -122.16...",POINT (-122.1621872388229 37.74082109234572),-122.162187,...,1,6.0,NaN,SEASIDE,14104.0,CA,93955,5.0,80358,879
7,174,06,001,06001430200,Census Tract 4302,0.000545,0.139575,"POLYGON ((-122.075277105 37.7207470070001, -12...",POINT (-122.0615788965973 37.72168112733675),-122.061579,...,1,3.0,2.0,MARINA,14302.0,CA,93933,6.0,199090,845
8,394,06,013,06013310000,Census Tract 3100,0.000203,0.101752,"POLYGON ((-121.917855117 38.0277880450001, -12...",POINT (-121.8917724088917 38.0264832798069),-121.891772,...,1,4.0,2.0,SAN FRANCISCO,33100.0,CA,94132,26.0,658122,1206
9,394,06,013,06013310000,Census Tract 3100,0.000203,0.101752,"POLYGON ((-121.917855117 38.0277880450001, -12...",POINT (-121.8917724088917 38.0264832798069),-121.891772,...,1,4.0,2.0,SAN FRANCISCO,33100.0,CA,94132,26.0,658122,1206


#### Filter out irregular trips and/or repeats or non work-home trips

In [ ]:
#selecting trips by people who go to school at least once

# place3 = school_home_locations.groupby('HHPER').filter(lambda x : x['place'].isin(["school"]).any())
# place3.head()



In [ ]:
def count_place(df):
    '''adds a column with each school trips numbered. ie: the first home trip by that person is 1, 
    the second home trip is 2, etc.'''
    return df.join(df['place'].str.get_dummies()
                  .cumsum().add_suffix('_count')).head(n=15)

trips = place4.groupby('HHPER', group_keys=False).apply(count_place)

trips.head()

In [ ]:
#eliminate records for persons that are never home

neverhome = trips[trips.home_count.isnull()]
neverathome = neverhome["HHPER"].unique()
trips1 = trips[~trips.HHPER.isin(neverathome)] 


# create a trip id for each trip that is 2 numbers where the first is the number of the school trip and the sceond
# is the number of the home trip

trips1["trip_ID"] = trips1["school_count"].map(str) + trips1["home_count"].map(str)
trips1["HHPERTRIP"] = trips1["HHPER"].map(str) + (trips1["trip_ID"]).map(str)
trips1["HHPERTRIP"] = trips1["HHPERTRIP"].map(float)/100
trips1['num_trips'] = trips1.groupby('HHPERTRIP').cumcount() + 1 #adds a column with the cumulative count of trips taken
#by that person that day. 12 is the max.
trips1.head()


In [ ]:
#isolate just the trips that are the start of home-school or school-home trip (place column is the start of the trip)
# home_school_trips = trips1.loc[trips['place'].isin(["home","school"])]
school_trips = trips1.loc[trips['place'].isin(["school"])]

school_trips.head()

Take just the trips where the place is not the same as the one before it and/or the person is not the same as the one before it. Either it is a different person or the destiation of the trip is different than the destination of the previous trip by the same person.


This captures 'from home' trips that end at work and 'from work' trips that end at home. Also, captures end-of-day 'from home' trips that don't end at work and 'from work' trips that don't end at home (for now, these will be filtered later).


In [ ]:
trips_clean = home_school_trips.loc[(home_school_trips.HHPER != home_school_trips.HHPER.shift(-1)) | (home_school_trips.place != home_school_trips.place.shift(-1))]

trips_clean.head()


Add a column for from mode and rename mode column to specify its the "to-mode"

In [ ]:
home_school_trips['mode_from'] = home_school_trips.groupby('HHPER', group_keys=False).MODE.shift(-1)
home_school_trips.rename({"SMODE": "to_mode"}, inplace=True, axis=1)
home_school_trips.head()


Get mode dummy columns

In [ ]:
hs_trips_mode = pd.get_dummies(home_school_trips, columns=["mode_from"])
hs_trips_mode.head()

### Add columns with dwell time at the origin and travel time to arrival location.

In [ ]:
place['act_start_time'] = place['ARR_HR'] + (place['ARR_MIN']/60)

place['act_end_time'] = place['DEP_HR'] + (place['DEP_MIN']/60)


## Step 4. Assign modes: drive alone, shared, bike, walk, walkTransitwalk, driveTransitwalk, walkTransitdrive.

In [30]:
tripsR['MODE'] = np.where((((tripsR['MODE1'].isin([5.0,6.0,7.0,8.0,9.0,10.0])) |
                                                           (tripsR['MODE2'].isin([5.0,6.0,7.0,8.0,9.0,10.0])) |
                                                           (tripsR['MODE3'].isin([5.0,6.0,7.0,8.0,9.0,10.0])) |
                                                           (tripsR['MODE4'].isin([5.0,6.0,7.0,8.0,9.0,10.0])) |
                                                           (tripsR['MODE5'].isin([5.0,6.0,7.0,8.0,9.0,10.0])) |
                                                           (tripsR['MODE6'].isin([5.0,6.0,7.0,8.0,9.0,10.0])) |
                                                           (tripsR['MODE7'].isin([5.0,6.0,7.0,8.0,9.0,10.0])) |
                                                           (tripsR['MODE8'].isin([5.0,6.0,7.0,8.0,9.0,10.0])) |
                                                           (tripsR['MODE9'].isin([5.0,6.0,7.0,8.0,9.0,10.0])) |
                                                           (tripsR['MODE10'].isin([5.0,6.0,7.0,8.0,9.0,10.0])) |
                                                           (tripsR['MODE11'].isin([5.0,6.0,7.0,8.0,9.0,10.0])) |
                                                           (tripsR['MODE12'].isin([5.0,6.0,7.0,8.0,9.0,10.0])) |
                                                           (tripsR['MODE13'].isin([5.0,6.0,7.0,8.0,9.0,10.0])) |
                                                           (tripsR['MODE14'].isin([5.0,6.0,7.0,8.0,9.0,10.0]))) &
                            ((tripsR['MODE1'].isin([15.0,16.0,17.0,19.0,20.0,21.0,22.0,23.0,24.0,25.0,26.0,27.0,28.0,29.0])) |
                             (tripsR['MODE2'].isin([15.0,16.0,17.0,19.0,20.0,21.0,22.0,23.0,24.0,25.0,26.0,27.0,28.0,29.0])) |
                             (tripsR['MODE3'].isin([15.0,16.0,17.0,19.0,20.0,21.0,22.0,23.0,24.0,25.0,26.0,27.0,28.0,29.0])) |
                             (tripsR['MODE4'].isin([15.0,16.0,17.0,19.0,20.0,21.0,22.0,23.0,24.0,25.0,26.0,27.0,28.0,29.0])) |
                             (tripsR['MODE5'].isin([15.0,16.0,17.0,19.0,20.0,21.0,22.0,23.0,24.0,25.0,26.0,27.0,28.0,29.0])) |
                             (tripsR['MODE6'].isin([15.0,16.0,17.0,19.0,20.0,21.0,22.0,23.0,24.0,25.0,26.0,27.0,28.0,29.0])) |
                             (tripsR['MODE7'].isin([15.0,16.0,17.0,19.0,20.0,21.0,22.0,23.0,24.0,25.0,26.0,27.0,28.0,29.0])) |
                             (tripsR['MODE8'].isin([15.0,16.0,17.0,19.0,20.0,21.0,22.0,23.0,24.0,25.0,26.0,27.0,28.0,29.0])) |
                             (tripsR['MODE9'].isin([15.0,16.0,17.0,19.0,20.0,21.0,22.0,23.0,24.0,25.0,26.0,27.0,28.0,29.0])) |
                             (tripsR['MODE10'].isin([15.0,16.0,17.0,19.0,20.0,21.0,22.0,23.0,24.0,25.0,26.0,27.0,28.0,29.0])) |
                             (tripsR['MODE11'].isin([15.0,16.0,17.0,19.0,20.0,21.0,22.0,23.0,24.0,25.0,26.0,27.0,28.0,29.0])) |
                             (tripsR['MODE12'].isin([15.0,16.0,17.0,19.0,20.0,21.0,22.0,23.0,24.0,25.0,26.0,27.0,28.0,29.0])) |
                             (tripsR['MODE13'].isin([15.0,16.0,17.0,19.0,20.0,21.0,22.0,23.0,24.0,25.0,26.0,27.0,28.0,29.0])) |
                             (tripsR['MODE14'].isin([15.0,16.0,17.0,19.0,20.0,21.0,22.0,23.0,24.0,25.0,26.0,27.0,28.0,29.0])))),
                          'drive_transit','no')

NameError: name 'tripsR' is not defined